In [38]:
from sklearn import svm

import matplotlib.pyplot as plt
import pandas as pd
import argparse
import numpy as np
import os
from copy import deepcopy
from tqdm.auto import tqdm
from dataloaders.dataloader import FCMatrixDataset
from utils import balanced_random_split_v2

from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.data.dataset import random_split
from torch.utils.data.sampler import SubsetRandomSampler

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix




import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim

In [68]:
mrmr_features = np.array([1140, 536, 223, 907, 1449, 499, 1293, 45, 135, 1440, 879, 1384, 1210, 1316, 122, 22, 492, 638, 765, 1027, 1464, 501, 1462, 395, 26, 1079, 70, 425, 1403, 1409, 1318, 886, 1459, 1448, 939, 1163, 547, 10, 413, 676, 131, 216, 942, 1136, 1386, 232, 1455, 1337, 814, 139, 392, 1376, 1382, 471, 656]
    )
# mrmr_features=None

ds = "data/csv/severe_rds.csv"
data_dir = "data/fetched/25751/raw"
udi = "25751"
dataset = FCMatrixDataset(ds, data_dir, udi, mapping=None, mrmr=mrmr_features)
labels = np.genfromtxt(ds)
eids = labels[:,0]
labels = labels[:,1]

np.random.seed(42)
batch_size = 32

total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = int(0.105 * total_size)
test_size = total_size - train_size - val_size

print(train_size, val_size, test_size)
train, val, test = balanced_random_split_v2(dataset, [train_size, val_size, test_size],num_classes=2)

# train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)

print(len(train), len(val), len(test))

pm = {
    'C': [0.01, 0.1, 1],
    'gamma': [1.0, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear', 'sigmoid'],
}

# pm = {
#     # 'C': [0.01, 0.1, 1],
#     # 'gamma': [1.0, 0.1, 0.01, 0.001],
#     # 'kernel': ['poly'],
#     # 'degree': [2, 3, 4, 5],
# }

pm = {
    'C' :[0.01],
    'gamma': [1.0],
    'kernel': ['linear'],
}
train_x = np.array([np.array(x) for x, y in train])
train_y = np.array([y for x, y in train])

test_x = np.array([np.array(x) for x, y in test])
test_y = np.array([y for x, y in test])
clf = svm.SVC(probability=True)
search = GridSearchCV(clf, pm, n_jobs=-1, cv=10, verbose=3)
search.fit(train_x, train_y)
clf = search.best_estimator_

y_pred = clf.predict(test_x)

# get confusion matrix
cm = confusion_matrix(test_y, y_pred)
acc = np.trace(cm) / np.sum(cm)


# n_perms = 100
# perm_scores = []
# for i in range(n_perms):
#     perm_y_test = np.random.permutation(test_y)
#     pred = clf.predict(test_x)
#     cm = confusion_matrix(perm_y_test, pred)
#     perm_acc = np.trace(cm) / np.sum(cm)
#     # print(f'{i}:', perm_acc)
#     perm_scores.append(perm_acc)

# num_better_scores = np.count_nonzero(np.array(perm_scores) >= acc)
# print('Number of better permuted scores:', num_better_scores)
# pval = (num_better_scores + 1) / (n_perms + 1)
# print('p-value:', pval)
print(acc)

print(search.best_params_)
print(search.cv_results_)



188 24 24
Length of class indices: [118, 118]
Length of subset idxs: [188, 24, 24]
188 24 24
Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV 1/10] END .C=0.01, gamma=1.0, kernel=linear;, score=0.842 total time=   0.0s
[CV 3/10] END .C=0.01, gamma=1.0, kernel=linear;, score=0.895 total time=   0.0s
[CV 2/10] END .C=0.01, gamma=1.0, kernel=linear;, score=0.684 total time=   0.0s
[CV 5/10] END .C=0.01, gamma=1.0, kernel=linear;, score=0.895 total time=   0.0s
[CV 6/10] END .C=0.01, gamma=1.0, kernel=linear;, score=0.789 total time=   0.0s
[CV 7/10] END .C=0.01, gamma=1.0, kernel=linear;, score=0.947 total time=   0.0s
[CV 10/10] END C=0.01, gamma=1.0, kernel=linear;, score=0.778 total time=   0.0s
[CV 4/10] END .C=0.01, gamma=1.0, kernel=linear;, score=0.632 total time=   0.0s
[CV 9/10] END .C=0.01, gamma=1.0, kernel=linear;, score=0.833 total time=   0.0s
[CV 8/10] END .C=0.01, gamma=1.0, kernel=linear;, score=0.684 total time=   0.0s
0.875
{'C': 0.01, 'gamma': 1.0, 'ker